In [1]:
import os
import sys
import tensorflow as tf
from tensorflow import gfile
from tensorflow import logging

import pickle
import numpy as np
import pprint

model_file = './inception_model/classify_image_graph_def.pb'
input_description_file = './dataset/flickr30k/results_20130124.token'
input_img_dir = './dataset/flickr30k-images/'
output_folder = './dataset/inception_v3_features_bakup/'

# 每1000张图片生成一个文件夹
batch_size = 1000

if not gfile.Exists(output_folder):
    gfile.MakeDirs(output_folder)
    

In [4]:
def parse_token_file(token_file):
    '''Parses input_description_file to a dict'''
    img_name_to_token = {}
    with gfile.GFile(token_file, 'r') as f:
        lines = f.readlines()
        
    for line in lines:
        img_id, description = line.strip('\n\r').split('\t')
        img_name = img_id.split('#')[0]
        img_name_to_token.setdefault(img_name, [])
        img_name_to_token[img_name].append(description)
        
    return img_name_to_token

img_name_to_token = parse_token_file(input_description_file)
all_img_names = img_name_to_token.keys()

logging.info("num of all images: %d", len(all_img_names))
pprint.pprint(list(all_img_names)[0:10])
pprint.pprint(img_name_to_token['10010052.jpg'])

INFO:tensorflow:num of all images: 31783
['1000092795.jpg',
 '10002456.jpg',
 '1000268201.jpg',
 '1000344755.jpg',
 '1000366164.jpg',
 '1000523639.jpg',
 '1000919630.jpg',
 '10010052.jpg',
 '1001465944.jpg',
 '1001545525.jpg']
['A girl is on rollerskates talking on her cellphone standing in a parking lot '
 '.\n',
 'A trendy girl talking on her cellphone while gliding slowly down the street '
 '.\n',
 'A young adult wearing rollerblades , holding a cellular phone to her ear .\n',
 'there is a young girl on her cellphone while skating .\n',
 'Woman talking on cellphone and wearing rollerskates .\n']


In [5]:
# 载入模型
def load_pretrain_inception_v3(model_file):
    with gfile.GFile(model_file, 'rb') as f:
        graph_def = tf.GraphDef() # 建立空的计算图
        graph_def.ParseFromString(f.read()) # 读取到计算图
        _ = tf.import_graph_def(graph_def, name = "")
        
load_pretrain_inception_v3(model_file)
        

In [9]:
# 计算有多少个子文件
num_batches = int(len(all_img_names) / batch_size)
if len(all_img_names) % batch_size != 0:
    num_batches += 1
    
with tf.Session() as sess:
    second_to_last_tensor = sess.graph.get_tensor_by_name("pool_3:0")
    for i in range(num_batches):
        batch_img_names = list(all_img_names)[i*batch_size : (i+1)*batch_size]
        batch_features = []
        for img_name in batch_img_names:
            img_path = os.path.join(input_img_dir, img_name)
            logging.info("processing img %s", img_name)
            if not gfile.Exists(img_path):
                continue
            img_data = gfile.FastGFile(img_path, 'rb').read()
            feature_vector = sess.run(
                second_to_last_tensor,
                feed_dict = {
                    "DecodeJpeg/contents:0": img_data
                })
            batch_features.append(feature_vector)
            
        batch_features = np.vstack(batch_features)
        output_filename = os.path.join(output_folder, "image_feature-%d.pickle" % i)
        logging.info("writing to file %s", output_filename)
        
        with gfile.GFile(output_filename, 'w') as f:
            pickle.dump((batch_img_names, batch_features), f)

INFO:tensorflow:processing img 1000092795.jpg
INFO:tensorflow:processing img 10002456.jpg
INFO:tensorflow:processing img 1000268201.jpg
INFO:tensorflow:processing img 1000344755.jpg
INFO:tensorflow:processing img 1000366164.jpg
INFO:tensorflow:processing img 1000523639.jpg
INFO:tensorflow:processing img 1000919630.jpg
INFO:tensorflow:processing img 10010052.jpg
INFO:tensorflow:processing img 1001465944.jpg
INFO:tensorflow:processing img 1001545525.jpg
INFO:tensorflow:processing img 1001573224.jpg
INFO:tensorflow:processing img 1001633352.jpg
INFO:tensorflow:processing img 1001773457.jpg
INFO:tensorflow:processing img 1001896054.jpg
INFO:tensorflow:processing img 100197432.jpg
INFO:tensorflow:processing img 100207720.jpg
INFO:tensorflow:processing img 1002674143.jpg
INFO:tensorflow:processing img 1003163366.jpg
INFO:tensorflow:processing img 1003420127.jpg
INFO:tensorflow:processing img 1003428081.jpg
INFO:tensorflow:processing img 100444898.jpg
INFO:tensorflow:processing img 1005216151

INFO:tensorflow:processing img 106027774.jpg
INFO:tensorflow:processing img 106027978.jpg
INFO:tensorflow:processing img 106039369.jpg
INFO:tensorflow:processing img 1060605297.jpg
INFO:tensorflow:processing img 1061467336.jpg
INFO:tensorflow:processing img 1061887159.jpg
INFO:tensorflow:processing img 106300392.jpg
INFO:tensorflow:processing img 106356264.jpg
INFO:tensorflow:processing img 10637120.jpg
INFO:tensorflow:processing img 106372282.jpg
INFO:tensorflow:processing img 1063866640.jpg
INFO:tensorflow:processing img 106490881.jpg
INFO:tensorflow:processing img 106514190.jpg
INFO:tensorflow:processing img 1065323785.jpg
INFO:tensorflow:processing img 1065831604.jpg
INFO:tensorflow:processing img 1066252238.jpg
INFO:tensorflow:processing img 1066831905.jpg
INFO:tensorflow:processing img 106691539.jpg
INFO:tensorflow:processing img 1067180831.jpg
INFO:tensorflow:processing img 1067675215.jpg
INFO:tensorflow:processing img 1067790824.jpg
INFO:tensorflow:processing img 1068427675.jpg

INFO:tensorflow:processing img 111811048.jpg
INFO:tensorflow:processing img 1118168796.jpg
INFO:tensorflow:processing img 1118557877.jpg
INFO:tensorflow:processing img 1118752484.jpg
INFO:tensorflow:processing img 1119015538.jpg
INFO:tensorflow:processing img 1119034454.jpg
INFO:tensorflow:processing img 111908743.jpg
INFO:tensorflow:processing img 1119418776.jpg
INFO:tensorflow:processing img 1119463452.jpg
INFO:tensorflow:processing img 11205420.jpg
INFO:tensorflow:processing img 1121053156.jpg
INFO:tensorflow:processing img 1121416483.jpg
INFO:tensorflow:processing img 11214205.jpg
INFO:tensorflow:processing img 11214370.jpg
INFO:tensorflow:processing img 11214470.jpg
INFO:tensorflow:processing img 112178718.jpg
INFO:tensorflow:processing img 11220707.jpg
INFO:tensorflow:processing img 1122177096.jpg
INFO:tensorflow:processing img 112220869.jpg
INFO:tensorflow:processing img 1122357785.jpg
INFO:tensorflow:processing img 112243673.jpg
INFO:tensorflow:processing img 112245524.jpg
INFO

INFO:tensorflow:processing img 117478651.jpg
INFO:tensorflow:processing img 1175268298.jpg
INFO:tensorflow:processing img 1176580356.jpg
INFO:tensorflow:processing img 1177949654.jpg
INFO:tensorflow:processing img 1177994172.jpg
INFO:tensorflow:processing img 1178705300.jpg
INFO:tensorflow:processing img 117883760.jpg
INFO:tensorflow:processing img 1179086155.jpg
INFO:tensorflow:processing img 117919518.jpg
INFO:tensorflow:processing img 1179226886.jpg
INFO:tensorflow:processing img 117956563.jpg
INFO:tensorflow:processing img 11808546.jpg
INFO:tensorflow:processing img 1181708011.jpg
INFO:tensorflow:processing img 118187095.jpg
INFO:tensorflow:processing img 1182733666.jpg
INFO:tensorflow:processing img 118309463.jpg
INFO:tensorflow:processing img 1184340890.jpg
INFO:tensorflow:processing img 1184440714.jpg
INFO:tensorflow:processing img 1184910489.jpg
INFO:tensorflow:processing img 1184967930.jpg
INFO:tensorflow:processing img 1186128541.jpg
INFO:tensorflow:processing img 1186391461.

INFO:tensorflow:processing img 1242394484.jpg
INFO:tensorflow:processing img 12424190.jpg
INFO:tensorflow:processing img 1242603438.jpg
INFO:tensorflow:processing img 1242629236.jpg
INFO:tensorflow:processing img 1243756.jpg
INFO:tensorflow:processing img 1243798805.jpg
INFO:tensorflow:processing img 124390903.jpg
INFO:tensorflow:processing img 1244140539.jpg
INFO:tensorflow:processing img 1244306891.jpg
INFO:tensorflow:processing img 1244485675.jpg
INFO:tensorflow:processing img 124488273.jpg
INFO:tensorflow:processing img 1245022983.jpg
INFO:tensorflow:processing img 1245194315.jpg
INFO:tensorflow:processing img 124523773.jpg
INFO:tensorflow:processing img 124534466.jpg
INFO:tensorflow:processing img 1246237064.jpg
INFO:tensorflow:processing img 1246239396.jpg
INFO:tensorflow:processing img 1246863003.jpg
INFO:tensorflow:processing img 1247181182.jpg
INFO:tensorflow:processing img 1247484195.jpg
INFO:tensorflow:processing img 1248357227.jpg
INFO:tensorflow:processing img 1248734482.j

...

INFO:tensorflow:processing img 8002838345.jpg
INFO:tensorflow:processing img 8003613232.jpg
INFO:tensorflow:processing img 800387501.jpg
INFO:tensorflow:processing img 8004084961.jpg
INFO:tensorflow:processing img 8004090394.jpg
INFO:tensorflow:processing img 8005292930.jpg
INFO:tensorflow:processing img 8006241096.jpg
INFO:tensorflow:processing img 8006608256.jpg
INFO:tensorflow:processing img 8008558496.jpg
INFO:tensorflow:processing img 8008888211.jpg
INFO:tensorflow:processing img 8011169691.jpg
INFO:tensorflow:processing img 8013541498.jpg
INFO:tensorflow:processing img 80143638.jpg
INFO:tensorflow:processing img 8015752233.jpg
INFO:tensorflow:processing img 801607443.jpg
INFO:tensorflow:processing img 801672676.jpg
INFO:tensorflow:processing img 8016751611.jpg
INFO:tensorflow:processing img 8017202978.jpg
INFO:tensorflow:processing img 8018246519.jpg
INFO:tensorflow:processing img 8018744438.jpg
INFO:tensorflow:processing img 8018747908.jpg
INFO:tensorflow:writing to file ./datas

INFO:tensorflow:processing img 8142543165.jpg
INFO:tensorflow:processing img 8143343377.jpg
INFO:tensorflow:processing img 8144714172.jpg
INFO:tensorflow:processing img 8144894905.jpg
INFO:tensorflow:processing img 8146024600.jpg
INFO:tensorflow:processing img 8146089465.jpg
INFO:tensorflow:processing img 8146641506.jpg
INFO:tensorflow:processing img 8151603778.jpg
INFO:tensorflow:processing img 8154438289.jpg
INFO:tensorflow:processing img 8154663567.jpg
INFO:tensorflow:processing img 8154670501.jpg
INFO:tensorflow:processing img 8154705592.jpg
INFO:tensorflow:processing img 8154861953.jpg
INFO:tensorflow:processing img 8154926258.jpg
INFO:tensorflow:processing img 8155411493.jpg
INFO:tensorflow:processing img 8155650726.jpg
INFO:tensorflow:processing img 8155663151.jpg
INFO:tensorflow:processing img 8155761670.jpg
INFO:tensorflow:processing img 8156336385.jpg
INFO:tensorflow:processing img 8156520592.jpg
INFO:tensorflow:processing img 8158019748.jpg
INFO:tensorflow:processing img 815

INFO:tensorflow:processing img 842961005.jpg
INFO:tensorflow:processing img 843218288.jpg
INFO:tensorflow:processing img 843616798.jpg
INFO:tensorflow:processing img 843843809.jpg
INFO:tensorflow:processing img 844258711.jpg
INFO:tensorflow:processing img 8443156.jpg
INFO:tensorflow:processing img 845047334.jpg
INFO:tensorflow:processing img 84507882.jpg
INFO:tensorflow:processing img 84509412.jpg
INFO:tensorflow:processing img 84509831.jpg
INFO:tensorflow:processing img 84514007.jpg
INFO:tensorflow:processing img 8454235.jpg
INFO:tensorflow:processing img 845623658.jpg
INFO:tensorflow:processing img 845940422.jpg
INFO:tensorflow:processing img 846070499.jpg
INFO:tensorflow:processing img 846085364.jpg
INFO:tensorflow:processing img 846516819.jpg
INFO:tensorflow:processing img 84713990.jpg
INFO:tensorflow:processing img 847782643.jpg
INFO:tensorflow:processing img 848007330.jpg
INFO:tensorflow:processing img 848180689.jpg
INFO:tensorflow:processing img 84819693.jpg
INFO:tensorflow:proc

INFO:tensorflow:processing img 888517718.jpg
INFO:tensorflow:processing img 889751330.jpg
INFO:tensorflow:processing img 890734502.jpg
INFO:tensorflow:processing img 89202781.jpg
INFO:tensorflow:processing img 892340814.jpg
INFO:tensorflow:processing img 89245315.jpg
INFO:tensorflow:processing img 89253626.jpg
INFO:tensorflow:processing img 892870649.jpg
INFO:tensorflow:processing img 89332423.jpg
INFO:tensorflow:processing img 89353191.jpg
INFO:tensorflow:processing img 893808757.jpg
INFO:tensorflow:processing img 89404014.jpg
INFO:tensorflow:processing img 89407459.jpg
INFO:tensorflow:processing img 894928353.jpg
INFO:tensorflow:processing img 894964960.jpg
INFO:tensorflow:processing img 89505897.jpg
INFO:tensorflow:processing img 895359811.jpg
INFO:tensorflow:processing img 895502702.jpg
INFO:tensorflow:processing img 897406883.jpg
INFO:tensorflow:processing img 897621891.jpg
INFO:tensorflow:processing img 89779839.jpg
INFO:tensorflow:processing img 89787769.jpg
INFO:tensorflow:proc

INFO:tensorflow:processing img 97138308.jpg
INFO:tensorflow:processing img 97138973.jpg
INFO:tensorflow:processing img 97162852.jpg
INFO:tensorflow:processing img 97233789.jpg
INFO:tensorflow:processing img 97234558.jpg
INFO:tensorflow:processing img 972381743.jpg
INFO:tensorflow:processing img 9726060.jpg
INFO:tensorflow:processing img 973353650.jpg
INFO:tensorflow:processing img 973827791.jpg
INFO:tensorflow:processing img 97406261.jpg
INFO:tensorflow:processing img 974924582.jpg
INFO:tensorflow:processing img 97495047.jpg
INFO:tensorflow:processing img 975131015.jpg
INFO:tensorflow:processing img 97577988.jpg
INFO:tensorflow:processing img 976392326.jpg
INFO:tensorflow:processing img 976414321.jpg
INFO:tensorflow:processing img 976581518.jpg
INFO:tensorflow:processing img 976639041.jpg
INFO:tensorflow:processing img 97731718.jpg
INFO:tensorflow:processing img 977365989.jpg
INFO:tensorflow:processing img 97748313.jpg
INFO:tensorflow:processing img 977856234.jpg
INFO:tensorflow:proces